# Crypto Market Data Dashboard


In [20]:
!pip install plotly

In [21]:
import requests
import pandas as pd
import plotly.express as px

coins = ['bitcoin', 'ethereum', 'solana', 'cardano', 'dogecoin']
vs_currency = 'usd'
days = 365


coingecko free API access to get dataset

In [22]:
def get_historical_data(coin, vs_currency='usd', days=days):
    url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
    params = {'vs_currency': vs_currency, 'days': days}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    df['volume'] = [v[1] for v in data['total_volumes']]
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['coin'] = coin
    return df

# Combine all data into one DataFrame
df_all = pd.concat([get_historical_data(c) for c in coins], ignore_index=True)
df_all.head()


,timestamp,price,volume,coin
0,2024-11-09,76550.021385,5.068747e+10,bitcoin
1,2024-11-10,76630.226314,3.143091e+10,bitcoin
2,2024-11-11,80466.716548,9.347313e+10,bitcoin
3,2024-11-12,88637.420003,1.339598e+11,bitcoin
4,2024-11-13,88264.599746,1.523568e+11,bitcoin


In [23]:
df_all.shape

(1830, 4)

In [35]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   timestamp     1830 non-null   datetime64[ns]
 1   price         1830 non-null   float64       
 2   volume        1830 non-null   float64       
 3   coin          1830 non-null   object        
 4   daily_return  1825 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 71.6+ KB


In [37]:
df_all.describe()

,timestamp,price,volume,daily_return
count,1830,1830.000000,1.830000e+03,1825.000000
mean,2025-05-10 11:56:42.240437248,21121.675374,1.795812e+10,0.001283
min,2024-11-09 00:00:00,0.141982,2.873593e+08,-0.247318
25%,2025-02-08 00:00:00,0.668579,1.917069e+09,-0.022371
50%,2025-05-10 12:00:00,183.072271,6.868296e+09,0.000102
75%,2025-08-10 00:00:00,3853.706132,2.765780e+10,0.021057
max,2025-11-08 03:53:57,124773.508231,1.904603e+11,0.729316
std,NaN,40928.176979,2.342123e+10,0.047084


In [38]:
fig = px.line(
    df_all,
    x='timestamp',
    y='price',
    color='coin',
    title=f'{days}-Day Price Trends for Top Cryptocurrencies'
)
fig.show()



In [39]:
fig2 = px.line(
    df_all,
    x='timestamp',
    y='volume',
    color='coin',
    title=f'{days}-Day Volume Trends'
)
fig2.show()



In [27]:
df_all['daily_return'] = df_all.groupby('coin')['price'].pct_change()

In [28]:
volatility = df_all.groupby('coin')['daily_return'].std().sort_values(ascending=False)
volatility


,daily_return
coin,
cardano,0.063070
dogecoin,0.053924
solana,0.045761
ethereum,0.040006
bitcoin,0.022941


In [29]:
pivot_df = df_all.pivot(index='timestamp', columns='coin', values='price')
corr = pivot_df.corr()
corr

coin,bitcoin,cardano,dogecoin,ethereum,solana
coin,,,,,
bitcoin,1.000000,0.158785,0.004114,0.726307,0.415770
cardano,0.158785,1.000000,0.812103,0.483229,0.680156
dogecoin,0.004114,0.812103,1.000000,0.428127,0.800416
ethereum,0.726307,0.483229,0.428127,1.000000,0.755104
solana,0.415770,0.680156,0.800416,0.755104,1.000000


In [30]:
import plotly.figure_factory as ff
fig3 = ff.create_annotated_heatmap(
    z=corr.values,
    x=list(corr.columns),
    y=list(corr.index),
    showscale=True,
    colorscale='Viridis'
)
fig3.update_layout(title='Correlation Between Crypto Prices')
fig3.show()


In [31]:
import plotly.figure_factory as ff
fig3 = ff.create_annotated_heatmap(
    z=corr.values,
    x=list(corr.columns),
    y=list(corr.index),
    showscale=True,
    colorscale='Viridis'
)
fig3.update_layout(title='Correlation Between Crypto Prices')
fig3.show()


In [33]:
df_all.to_csv('crypto_historical_data.csv', index=False)